In [ ]:
# Import necessary libraries for building and training the neural network
import torch  # PyTorch for tensor operations and neural network computations
import matplotlib.pyplot as plt  # For creating visualisations and plots
from sklearn.datasets import load_wine  # Load the wine classification dataset
from sklearn.model_selection import train_test_split  # Split data into train/test sets
from sklearn.preprocessing import StandardScaler  # Normalise features to have mean=0, std=1

In [ ]:
# Load the Wine dataset from scikit-learn
# This dataset contains 13 chemical measurements from 178 wine samples
# across 3 different wine cultivars (classes 0, 1, and 2)
X, y = load_wine(return_X_y=True, as_frame=True)

# Split the dataset into 70% training and 30% testing sets
# random_state ensures reproducibility of the split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.30, random_state=42
)

# Standardise the features to have mean=0 and standard deviation=1
# This is crucial for neural networks as it:
# 1. Helps gradient descent converge faster
# 2. Prevents features with larger scales from dominating the learning
# 3. Keeps activations in a reasonable range
scaler = StandardScaler().set_output(transform="pandas")
scaled_X_train = scaler.fit_transform(X_train)  # Fit scaler on training data and transform
scaled_X_test = scaler.transform(X_test)  # Transform test data using training statistics

In [ ]:
import seaborn as sns
import numpy as np

# Visualise feature correlations to understand relationships between chemical measurements
# Strong correlations may indicate redundant features or multicollinearity

# Combine scaled features with target labels for analysis
train_data = scaled_X_train.copy()
train_data['target'] = y_train.values

# Compute the correlation matrix for all features
# Values range from -1 (perfect negative correlation) to +1 (perfect positive correlation)
corr_matrix = train_data.drop('target', axis=1).corr()

plt.figure(figsize=(10, 8))

# Create a mask to hide the upper triangle (matrix is symmetric)
# This makes the heatmap cleaner and easier to read
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))

# Plot the correlation heatmap
# - annot=True displays correlation values in each cell
# - fmt=".2f" formats values to 2 decimal places
# - cmap='viridis' uses consistent colourmap with other plots
sns.heatmap(
    corr_matrix, 
    mask=mask, 
    annot=True, 
    fmt=".2f", 
    cmap='viridis', 
    vmin=-1, 
    vmax=1
)

plt.title('Feature Correlation Matrix')
plt.tight_layout()  # Adjust spacing for better appearance
plt.savefig('../figures/wine_correlation_matrix.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
from sklearn.decomposition import PCA

# Perform Principal Component Analysis (PCA) to visualise high-dimensional data in 2D
# PCA finds the directions of maximum variance in the data
# This helps us understand if the 3 wine classes are linearly separable

# Reduce the 13-dimensional feature space to 2 principal components
pca = PCA(n_components=2)
X_pca = pca.fit_transform(scaled_X_train)

# Create the PCA visualisation
plt.figure(figsize=(8, 6))

# Scatter plot with wine classes coloured differently
# - c=y_train colours points by their class label (0, 1, or 2)
# - cmap='viridis' uses consistent colourmap
# - edgecolor='k' adds black borders to make points more visible
# - alpha=0.7 adds slight transparency for overlapping points
scatter = plt.scatter(
    X_pca[:, 0], 
    X_pca[:, 1], 
    c=y_train, 
    cmap='viridis', 
    edgecolors='k', 
    alpha=0.7
)

# Label axes with variance explained by each principal component
# This tells us how much information each axis captures
plt.xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.2%} Variance)')
plt.ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.2%} Variance)')
plt.title('PCA Projection of Wine Dataset')

# Add a legend showing which colour corresponds to which wine class
plt.legend(*scatter.legend_elements(), title="Wine Class")

# Add a subtle grid for easier reading
plt.grid(True, alpha=0.3)

plt.tight_layout()  # Adjust spacing for better appearance
plt.savefig('../figures/wine_pca.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Visualise pairwise relationships between selected features
# A pairplot shows scatter plots for each feature pair and distributions on the diagonal
# This helps identify which feature combinations best separate the wine classes

# Select a subset of features that show strong class separation
# (Chosen based on the correlation matrix and domain knowledge)
features_of_interest = ['alcohol', 'flavanoids', 'color_intensity', 'proline']
subset_data = train_data[features_of_interest + ['target']]

# Create pairplot with:
# - hue='target' colours points by wine class
# - palette='viridis' for consistent colour scheme
# - corner=True only shows lower triangle (reduces redundancy)
# - diag_kind='kde' shows probability density on diagonal
sns.pairplot(
    subset_data, 
    hue='target', 
    palette='viridis', 
    corner=True, 
    diag_kind='kde'
)

plt.suptitle("Pairwise Feature Relationships", y=1.02)
plt.savefig('../figures/wine_pairplot.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
from pandas.plotting import parallel_coordinates

# Visualise all features simultaneously using parallel coordinates
# Each vertical line represents one feature, and each coloured line represents one wine sample
# This plot helps identify which features contribute most to class separation

plt.figure(figsize=(12, 6))

# Create parallel coordinates plot
# - Each wine sample is a line connecting its feature values
# - Lines are coloured by wine class (target)
# - colormap='viridis' for consistent styling
# - alpha=0.5 adds transparency to see overlapping patterns
parallel_coordinates(
    train_data, 
    'target', 
    colormap='viridis', 
    alpha=0.5
)

plt.title("Multivariate Feature Profiles")
plt.xticks(rotation=45)  # Rotate feature names for readability
plt.grid(True, alpha=0.3)  # Add subtle grid

plt.tight_layout()  # Adjust spacing for better appearance
plt.savefig('../figures/wine_parallel_coordinates.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# Select the best available computing device (GPU if available, otherwise CPU)
# GPUs can significantly speed up neural network training through parallel processing
device = torch.device(
    "cuda" if torch.cuda.is_available() 
    else "mps" if torch.backends.mps.is_available() 
    else "cpu"
)
print(f"Using device: {device}")

# Convert NumPy arrays to PyTorch tensors and move to device
# float32 is used for efficiency (GPUs are optimised for 32-bit floats)
# long is used for class labels in multi-class classification
X_train_t = torch.tensor(scaled_X_train.values, dtype=torch.float32).to(device)
y_train_t = torch.tensor(y_train.values, dtype=torch.long).to(device)
X_test_t = torch.tensor(scaled_X_test.values, dtype=torch.float32).to(device)
y_test_t = torch.tensor(y_test.values, dtype=torch.long).to(device)

# Define a simple Multi-Layer Perceptron (MLP) architecture
class SimpleMLP(nn.Module):
    """
    A simple feedforward neural network with one hidden layer.
    
    Architecture:
        Input Layer (13 features) -> Hidden Layer (64 neurones, ReLU) -> Output Layer (3 classes)
    
    Args:
        n_input (int): Number of input features
        n_hidden (int): Number of neurones in the hidden layer
        n_output (int): Number of output classes
    """
    def __init__(self, n_input, n_hidden, n_output):
        super(SimpleMLP, self).__init__()
        # First layer: transforms input features to hidden layer
        self.layer1 = nn.Linear(n_input, n_hidden)
        # ReLU activation introduces non-linearity
        # ReLU(x) = max(0, x) - simple but effective
        self.relu = nn.ReLU()
        # Second layer: transforms hidden layer to output logits
        self.layer2 = nn.Linear(n_hidden, n_output)

    def forward(self, x):
        """
        Forward pass through the network.
        
        Args:
            x (torch.Tensor): Input data
            
        Returns:
            torch.Tensor: Raw output scores (logits) for each class
        """
        # Pass through first layer, apply ReLU, then pass through second layer
        return self.layer2(self.relu(self.layer1(x)))

# Create model with:
# - 13 input features (chemical measurements)
# - 64 hidden neurones (enough capacity to learn wine patterns)
# - 3 output classes (three wine cultivars)
model = SimpleMLP(13, 64, 3).to(device)

# Define loss function and optimiser
# CrossEntropyLoss combines softmax and negative log-likelihood
# It's the standard loss for multi-class classification
criterion = nn.CrossEntropyLoss()

# Adam optimiser adapts learning rate for each parameter
# lr=0.01 is a good starting learning rate for most problems
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Training loop
train_losses = []

for epoch in range(20000):
    model.train()  # Set model to training mode
    
    # Forward pass: compute predictions
    logits = model(X_train_t)
    
    # Compute loss: how far are predictions from true labels?
    loss = criterion(logits, y_train_t)
    
    # Backward pass: compute gradients
    optimizer.zero_grad()  # Clear previous gradients
    loss.backward()  # Compute new gradients via backpropagation
    optimizer.step()  # Update weights using gradients
    
    # Store loss for plotting
    train_losses.append(loss.item())
    
    # Print progress every 5000 epochs
    if epoch % 5000 == 0:
        model.eval()  # Set model to evaluation mode
        with torch.no_grad():  # Disable gradient computation for efficiency
            test_logits = model(X_test_t)
            # Get predictions by selecting class with highest score
            predictions = torch.argmax(test_logits, dim=1)
            # Calculate accuracy
            acc = (predictions == y_test_t).float().mean()
            print(f"Epoch {epoch:5d} | Loss: {loss.item():.8f} | Test Acc: {acc:.1%}")

# Final evaluation on test set
model.eval()
with torch.no_grad():
    final_preds = torch.argmax(model(X_test_t), dim=1)
    final_acc = (final_preds == y_test_t).float().mean()

print(f"\nFinal Test Accuracy: {final_acc:.2%}")

In [ ]:
# Visualise the training loss over time to assess learning progress
# A decreasing loss indicates the model is successfully learning the patterns

plt.figure(figsize=(8, 5))
plt.plot(train_losses, label='Training Loss')  # Plot loss values for each epoch
plt.xlabel('Epoch')  # X-axis: training iteration number
plt.ylabel('Cross-Entropy Loss')  # Y-axis: loss value

# Use logarithmic scale on y-axis to better visualise exponential decay
# This makes it easier to see improvements even when loss becomes very small
plt.yscale('log')

# Add a subtle grid for easier reading of values
plt.grid(True, alpha=0.3)
plt.legend()

plt.tight_layout()  # Adjust spacing for better appearance
plt.savefig('../figures/wine_training_loss.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, roc_curve, auc
from sklearn.preprocessing import label_binarize

# Prepare predictions and probabilities for evaluation
# We need both hard predictions (class labels) and soft predictions (probabilities)
# for different types of analysis

# Ensure the model is in evaluation mode (disables dropout, batch norm, etc.)
model.eval()

# Compute predictions without tracking gradients (saves memory and computation)
with torch.no_grad():
    # Get raw output scores (logits) from the model
    test_logits = model(X_test_t)
    
    # Convert logits to probabilities using softmax
    # Softmax ensures probabilities sum to 1: p_i = exp(logit_i) / sum(exp(logit_j))
    test_probs = torch.softmax(test_logits, dim=1)
    
    # Get hard predictions by selecting the class with highest probability
    test_preds = torch.argmax(test_logits, dim=1)

# Move predictions from GPU/device memory to CPU for use with scikit-learn
# scikit-learn expects NumPy arrays, not PyTorch tensors
y_true = y_test_t.cpu().numpy()  # True labels
y_pred = test_preds.cpu().numpy()  # Predicted labels
y_prob = test_probs.cpu().numpy()  # Class probabilities

In [ ]:
# Create a confusion matrix to visualise classification performance
# The confusion matrix shows how often each class is correctly or incorrectly classified
# Diagonal elements = correct predictions, off-diagonal = mistakes

# Compute the confusion matrix
# cm[i,j] = number of samples with true class i predicted as class j
cm = confusion_matrix(y_true, y_pred)

# Get class names from the dataset for better labelling
class_labels = load_wine().target_names

plt.figure(figsize=(7, 6))

# Plot confusion matrix as a heatmap
# - annot=True displays count values in each cell
# - fmt='d' formats values as integers
# - cmap='viridis' uses consistent colour scheme
sns.heatmap(
    cm, 
    annot=True, 
    fmt='d', 
    cmap='viridis',
    xticklabels=class_labels,
    yticklabels=class_labels
)

plt.title('Confusion Matrix')
plt.ylabel('True Class')
plt.xlabel('Predicted Class')

plt.tight_layout()  # Adjust spacing for better appearance
plt.savefig('../figures/wine_confusion_matrix.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Plot ROC curves for multi-class classification
# ROC (Receiver Operating Characteristic) curves show the trade-off between
# true positive rate and false positive rate at different classification thresholds
# AUC (Area Under Curve) summarises performance: 1.0 = perfect, 0.5 = random

# Convert true labels to binary format for one-vs-rest ROC analysis
# For class 0: [1, 0, 0], for class 1: [0, 1, 0], etc.
y_true_bin = label_binarize(y_true, classes=[0, 1, 2])
n_classes = y_true_bin.shape[1]

# Storage for ROC curve data for each class
fpr = dict()  # False positive rate
tpr = dict()  # True positive rate
roc_auc = dict()  # Area under the ROC curve

# Compute ROC curve for each wine class
for i in range(n_classes):
    # Compare binary true labels vs. predicted probabilities for this class
    fpr[i], tpr[i], _ = roc_curve(y_true_bin[:, i], y_prob[:, i])
    # Calculate area under the ROC curve (higher is better)
    roc_auc[i] = auc(fpr[i], tpr[i])

# Create the ROC curve plot
plt.figure(figsize=(8, 6))

# Plot ROC curve for each class
colours = ['#440154', '#21918c', '#fde724']  # Viridis-inspired colours
for i in range(n_classes):
    plt.plot(
        fpr[i],
        tpr[i],
        color=colours[i],
        lw=2,
        label=f'{class_labels[i]} (AUC = {roc_auc[i]:.2f})',
    )

# Add diagonal reference line (random classifier baseline)
# A random classifier has AUC = 0.5
plt.plot([0, 1], [0, 1], color='gray', lw=1, linestyle='--', label='Random')

plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curves (One-vs-Rest)')
plt.legend(loc="lower right")
plt.grid(True, alpha=0.3)  # Add subtle grid

plt.tight_layout()  # Adjust spacing for better appearance
plt.savefig('../figures/wine_roc_curves.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Visualise hidden layer activations to understand what the network learnt
# Each of the 64 hidden neurones learns to detect different chemical patterns
# By examining activations, we can see how the network internally represents different wines

# Extract hidden layer activations by using a forward hook
# A hook allows us to capture intermediate layer outputs during forward pass
hidden_activations = []

def hook_fn(module, input, output):
    """Hook function to capture ReLU output (hidden layer activations)"""
    hidden_activations.append(output.detach().cpu())

# Register the hook on the ReLU layer (which outputs the hidden layer activations)
hook = model.relu.register_forward_hook(hook_fn)

# Perform forward pass to trigger the hook and collect activations
model.eval()
with torch.no_grad():
    _ = model(X_test_t)

# Remove the hook (good practice to clean up)
hook.remove()

# Extract activations as a numpy array
# Shape: (n_test_samples, n_hidden_neurones)
activations = hidden_activations[0].numpy()

# Convert labels to numpy for easier indexing
y_test_np = y_test_t.cpu().numpy()

print(f"Activation matrix shape: {activations.shape}")
print(f"Hidden layer has {activations.shape[1]} neurones")
print(f"Test set has {activations.shape[0]} samples")

In [ ]:
# Create a comprehensive heatmap showing activation patterns for all wines
# This reveals which neurones are most active for each wine class

# Sort samples by class for clearer visualisation
sort_indices = np.argsort(y_test_np)
sorted_activations = activations[sort_indices]
sorted_labels = y_test_np[sort_indices]

# Find boundaries between classes
class_0_end = np.sum(sorted_labels == 0)
class_1_end = class_0_end + np.sum(sorted_labels == 1)

plt.figure(figsize=(12, 8))

# Create heatmap where:
# - Each column is a test sample (wine)
# - Each row is a hidden neurone
# - Colour intensity shows activation strength (ReLU output, so 0 or positive)
plt.imshow(sorted_activations.T, aspect='auto', cmap='viridis', interpolation='nearest')
cbar = plt.colorbar(label='Activation Level')

# Add vertical lines to separate the three wine classes
plt.axvline(x=class_0_end - 0.5, color='red', linestyle='--', linewidth=2)
plt.axvline(x=class_1_end - 0.5, color='red', linestyle='--', linewidth=2)

# Add class labels at the top
class_0_centre = class_0_end / 2
class_1_centre = class_0_end + (class_1_end - class_0_end) / 2
class_2_centre = class_1_end + (len(sorted_labels) - class_1_end) / 2

plt.text(class_0_centre, -3, class_labels[0], ha='center', fontsize=12, fontweight='bold')
plt.text(class_1_centre, -3, class_labels[1], ha='center', fontsize=12, fontweight='bold')
plt.text(class_2_centre, -3, class_labels[2], ha='center', fontsize=12, fontweight='bold')

plt.xlabel('Test Samples (sorted by wine class)')
plt.ylabel('Hidden Neurone')
plt.title('Hidden Layer Activation Heatmap\nEach wine class activates different neurone patterns')

plt.tight_layout()
plt.savefig('../figures/wine_activation_heatmap.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Analyse which neurones are most important for each wine class
# By averaging activations per class, we can identify class-specific neurones

# Compute average activation for each class
avg_activations = np.zeros((3, activations.shape[1]))
for class_idx in range(3):
    class_mask = y_test_np == class_idx
    avg_activations[class_idx] = activations[class_mask].mean(axis=0)

# Create bar plot showing average activation per class
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

for class_idx in range(3):
    ax = axes[class_idx]
    
    # Create bar plot of average neurone activations for this wine class
    bars = ax.bar(
        range(64), 
        avg_activations[class_idx],
        color='#440154',  # Viridis dark colour
        alpha=0.7,
        edgecolor='k'
    )
    
    # Highlight the top 5 most active neurones for this class
    top_5_neurones = np.argsort(avg_activations[class_idx])[-5:]
    for neurone_idx in top_5_neurones:
        bars[neurone_idx].set_color('#fde724')  # Viridis bright colour
    
    ax.set_xlabel('Hidden Neurone')
    ax.set_ylabel('Average Activation')
    ax.set_title(f'{class_labels[class_idx]}\n(Top 5 neurones highlighted)')
    ax.grid(True, alpha=0.3, axis='y')
    ax.set_xlim(-1, 64)

plt.suptitle('Average Hidden Layer Activation by Wine Class', fontsize=14, y=1.02)
plt.tight_layout()
plt.savefig('../figures/wine_avg_activation_by_class.png', dpi=300, bbox_inches='tight')
plt.show()

# Print the top 5 most discriminative neurones for each class
print("Top 5 most active neurones for each wine class:\n")
for class_idx in range(3):
    top_neurones = np.argsort(avg_activations[class_idx])[-5:][::-1]
    print(f"{class_labels[class_idx]}:")
    for rank, neurone_idx in enumerate(top_neurones, 1):
        activation_val = avg_activations[class_idx, neurone_idx]
        print(f"  {rank}. Neurone {neurone_idx:2d}: {activation_val:.3f}")
    print()